In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


df = pd.read_csv('/Users/cstone/Documents/Projects/MLB_Pitch_Prediction/data/raw/blake_snell.csv')

df.head()


,Unnamed: 0,pitch_type,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,at_bat_number,...,bat_score,fld_score,delta_home_win_exp,delta_run_exp,batter_is_right,pitcher_is_right,inning_top,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_Strategic
0,0,CU,0,2,0.0,1.0,1.0,2,6,47,...,0,1,-0.077,-0.311,1,0,0,True,False,False
1,1,FF,0,1,0.0,1.0,1.0,2,6,47,...,0,1,0.000,-0.098,1,0,0,True,False,False
2,2,FF,0,0,0.0,1.0,1.0,2,6,47,...,0,1,0.000,-0.063,1,0,0,True,False,False
3,3,CH,1,0,0.0,0.0,1.0,2,6,46,...,0,1,0.035,0.284,1,0,0,True,False,False
4,4,CU,0,0,0.0,0.0,1.0,2,6,46,...,0,1,0.000,0.025,1,0,0,True,False,False


In [8]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler




Train test split

In [9]:
X = df.loc[:, df.columns!='pitch_type']
y = df['pitch_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

Scale the data

In [11]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)